# Projet : Plan your trip with Kayak

<img src="img/kayak.png" width="900">

L'équipe marketing a besoin d'aide pour un nouveau projet. Après avoir effectué des recherches auprès des utilisateurs, l'équipe a découvert que 70 % de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d'informations sur la destination vers laquelle ils se rendent.

L'équipe marketing de Kayak souhaite donc créer une application qui recommandera aux gens où planifier leurs prochaines vacances. L'application devrait être basée sur des données réelles concernant la météo et les hôtels de la région.
 
L'application devrait alors être en mesure de recommander les meilleures destinations et hôtels en fonction des variables ci-dessus à tout moment.

Comme le projet vient juste de démarrer, l'équipe ne dispose d'aucune donnée pouvant être utilisée pour créer cette application. Par conséquent, voici les tâches à réaliser :

- Extraire les données des destinations
- Obtenir des données météorologiques pour chaque destination
- Obtenir des informations sur les hôtels de chaque destination
- Stocker toutes les informations ci-dessus dans un data lake
- Extraire, transformer et charger les données nettoyées du data lake vers un data warehouse
- Deux cartes avec un Top 5 des destinations et un Top 20 des hôtels de la région

### Import des bibliothèques

In [3]:
import pandas as pd
import requests
import plotly_express as px
import plotly.graph_objects as go
import boto3
from sqlalchemy import create_engine, text
from keys import API_key, aws_access_key_id, aws_secret_access_key, PASSWORD

L'équipe marketing souhaite d'abord se concentrer sur les meilleures destinations à visiter en France.  
Selon One Week In.com, voici les 35 meilleures destinations à visiter en France :

In [6]:
destinations = ["Mont Saint Michel",
                    "St Malo",
                    "Bayeux",
                    "Le Havre",
                    "Rouen",
                    "Paris",
                    "Amiens",
                    "Lille",
                    "Strasbourg",
                    "Chateau du Haut Koenigsbourg",
                    "Colmar",
                    "Eguisheim",
                    "Besancon",
                    "Dijon",
                    "Annecy",
                    "Grenoble",
                    "Lyon",
                    "Gorges du Verdon",
                    "Bormes les Mimosas",
                    "Cassis",
                    "Marseille",
                    "Aix en Provence",
                    "Avignon",
                    "Uzes",
                    "Nimes",
                    "Aigues Mortes",
                    "Saintes Maries de la mer",
                    "Collioure",
                    "Carcassonne",
                    "Ariege",
                    "Toulouse",
                    "Montauban",
                    "Biarritz",
                    "Bayonne",
                    "La Rochelle"]

## Coordonnées GPS des destinations avec l'API Nominatim


In [ ]:
# URL de l'API Nominatim
url = "https://nominatim.openstreetmap.org/search"

# En-têtes de la requête
headers = {
    'User-Agent': 'Chrome/130.0'
}

# Liste pour stocker les données
data_list = []

# Boucle sur chaque destination
for destination in destinations:
    params = {
        "q": destination,  
        "format": "json",  
        'countrycodes': 'FR', 
        "limit": 1  
    }

    # Requête vers l'API
    response = requests.get(url, params=params, headers=headers) 

    if response.status_code == 200:
        data = response.json()  
        if data: 
            lat = float(data[0]["lat"])
            lon = float(data[0]["lon"])
            data_list.append({"Destination":destination, "Latitude": lat, "Longitude": lon})
        else:
            print(f"Aucun résultat trouvé pour {destination}")
    else:
        print(f"Erreur {response.status_code} pour {destination}")

# Création d'un DataFrame 
df = pd.DataFrame(data_list).reset_index(names="Id")
df


,Id,Destination,Latitude,Longitude
0,0,Mont Saint Michel,48.635954,-1.511460
1,1,St Malo,48.649518,-2.026041
2,2,Bayeux,49.276462,-0.702474
3,3,Le Havre,49.493898,0.107973
4,4,Rouen,49.440459,1.093966
5,5,Paris,48.853495,2.348391
6,6,Amiens,49.894171,2.295695
7,7,Lille,50.636565,3.063528
8,8,Strasbourg,48.584614,7.750713
9,9,Chateau du Haut Koenigsbourg,48.249411,7.344320


##  Météo du jour et des 7 suivants avec OpenWeatherMap

In [12]:
# URL de l'API OpenWeatherMap
url = "https://api.openweathermap.org/data/3.0/onecall"

for index in df.index:
    lat = df.loc[index,'Latitude']
    lon = df.loc[index,'Longitude']

    params = {
        "lat": lat,
        "lon": lon,
        "appid": API_key,
        "lang": "fr",
        "units": "metric",
        "exclude": "current,minutely,hourly,alerts"
    }

    response = requests.get(url, params=params)
    data = response.json()


    for i in range(8):  # Boucle jour actuel + 7 jours suivants
        temp = data['daily'][i]['temp']['day'] 
        try: 
            data['daily'][i]['rain']
        except:
            rain = 0
        else:
            rain = data['daily'][i]['rain']
        df.loc[index, f"Temp_J{i}"] = temp
        df.loc[index, f"Rain_J{i}"] = rain

df

,Id,Destination,Latitude,Longitude,Temp_J0,Rain_J0,Temp_J1,Rain_J1,Temp_J2,Rain_J2,Temp_J3,Rain_J3,Temp_J4,Rain_J4,Temp_J5,Rain_J5,Temp_J6,Rain_J6,Temp_J7,Rain_J7
0,0,Mont Saint Michel,48.635954,-1.511460,8.78,0.00,9.03,0.00,10.08,0.00,11.14,0.00,13.47,0.0,13.36,0.00,12.46,0.00,15.48,0.00
1,1,St Malo,48.649518,-2.026041,8.08,0.00,7.79,0.00,8.30,0.00,8.95,0.00,11.16,0.0,12.92,0.00,13.33,0.00,14.56,0.27
2,2,Bayeux,49.276462,-0.702474,7.45,0.00,7.29,0.00,8.50,0.00,9.28,0.00,12.76,0.0,13.99,0.00,15.07,0.00,15.74,0.00
3,3,Le Havre,49.493898,0.107973,7.94,0.00,8.31,0.00,8.76,0.00,9.56,0.00,11.17,0.0,12.29,0.00,11.57,0.00,13.08,0.00
4,4,Rouen,49.440459,1.093966,7.92,0.00,9.06,0.00,9.77,0.00,11.23,0.00,13.29,0.0,14.74,0.00,14.73,0.00,15.82,0.00
5,5,Paris,48.853495,2.348391,8.46,0.00,9.37,0.00,9.72,0.00,11.58,0.00,13.25,0.0,15.35,0.00,14.73,0.00,15.27,0.00
6,6,Amiens,49.894171,2.295695,7.83,0.00,8.20,0.00,9.14,0.00,10.66,0.00,12.87,0.0,14.44,0.00,15.07,0.00,15.49,0.00
7,7,Lille,50.636565,3.063528,7.68,0.00,8.31,0.00,8.03,0.00,9.24,0.00,12.71,0.0,13.51,0.00,14.96,0.00,15.56,0.00
8,8,Strasbourg,48.584614,7.750713,7.34,0.00,7.79,0.00,7.44,0.00,8.43,0.00,10.91,0.0,12.53,0.00,13.39,0.00,14.25,0.00
9,9,Chateau du Haut Koenigsbourg,48.249411,7.344320,3.72,0.00,4.22,0.00,6.42,0.00,7.56,0.00,10.05,0.0,12.22,0.00,11.95,0.00,12.81,0.00


In [13]:
# Enregistrement au format csv
df.to_csv('src/meteo.csv', index=False, header=True)

## Web scraping de Booking.com pour obtenir les meilleurs hôtels 

In [18]:
!python booking.py "{';'.join([d.replace(' ', '%') for d in destinations])}"

2025-03-01 20:16:23 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-03-01 20:16:24 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.1, Platform Windows-11-10.0.26100-SP0
2025-03-01 20:16:24 [scrapy.addons] INFO: Enabled addons:
[]
2025-03-01 20:16:24 [scrapy.extensions.telnet] INFO: Telnet Password: 4b3e4cb982a43335
2025-03-01 20:16:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-03-01 20:16:24 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/130.0'}
2025-03-01 20:16:25 [scrapy.middleware] INFO: Enabled downloader middlewares:
['s

In [ ]:
# Création d'un dataframe avec les données scrapées
hotels = pd.read_json("src/best_hotels.json")
hotels = hotels.sort_values(by=['destination','score'],ascending=[True,False])

In [ ]:
# 20 meilleurs hôtels par destination
best_hotels = hotels.groupby('destination').head(20)
best_hotels

,destination,nom,score,latitude,longitude,url,description
449,Aigues Mortes,Boutique Hôtel des Remparts & Spa,"9,3",43.568036,4.190344,https://www.booking.com/hotel/fr/les-remparts-...,Aménagé dans une ancienne base militaire datan...
723,Aigues Mortes,La Villa Mazarin,"9,2",43.564987,4.191752,https://www.booking.com/hotel/fr/la-villa-maza...,Description de l’établissement La Villa Mazari...
717,Aigues Mortes,GOLF HOTEL,"9,1",43.568057,4.102608,https://www.booking.com/hotel/fr/golf-la-grand...,"Situé à La Grande-Motte, à 2 km de la plage de..."
719,Aigues Mortes,Hotel Les Acacias,"9,1",43.538658,4.134977,https://www.booking.com/hotel/fr/les-acacias-l...,"L'Hotel Les Acacias, de style provençal, situé..."
725,Aigues Mortes,La Maison de Lyna,"9,1",43.566018,4.192353,https://www.booking.com/hotel/fr/la-maison-de-...,L’établissement La Maison de Lyna vous accueil...
...,...,...,...,...,...,...,...
806,Uzes,La Taverne,"8,6",44.014659,4.420051,https://www.booking.com/hotel/fr/la-taverne.fr...,"La Taverne est située à Uzès, dans une rue pié..."
804,Uzes,La Maison d'Uzès Relais & Châteaux,"8,5",44.012744,4.421131,https://www.booking.com/hotel/fr/la-maison-d-u...,Situé dans le centre de la vieille ville d'Uzè...
779,Uzes,Hotel Restaurant Le Clos De Pradines,"8,4",44.046198,4.443198,https://www.booking.com/hotel/fr/clos-des-prad...,L'établissement Le Clos de Pradines vous accue...
805,Uzes,Patio De Violette,"8,4",44.028431,4.420024,https://www.booking.com/hotel/fr/patio-de-viol...,Situé dans la campagne du Languedoc-Roussillon...


In [ ]:
# Enregistrement au format csv
best_hotels.to_csv('src/best_hotels.csv', index=False, header=True)

## Stockage des informations dans un datalake S3

In [ ]:
# Création d'un session
session = boto3.Session(aws_access_key_id = aws_access_key_id, 
                        aws_secret_access_key = aws_secret_access_key,
                        region_name="eu-west-3")
# Initialisation de la ressource avec la session
s3 = session.resource('s3')

In [ ]:
# Création d'un bucket
bucket = s3.create_bucket(Bucket="bucket-plan-your-trip-with-kayak",
                          CreateBucketConfiguration={'LocationConstraint':'eu-west-3'})

In [9]:
# Envoie des fichiers dans le bucket
bucket.upload_file('src/best_hotels.csv', 'best_hotels.csv')
bucket.upload_file('src/meteo.csv', 'meteo.csv') 

In [ ]:
# Vérification
for obj in bucket.objects.all():
    print(obj.key)

best_hotels.csv
meteo.csv


<img src="img/bucket.png" width="900">

## Data warehouse : création d'une BDD dans RDS

#### ETL-Extract : Récupération des données dans le S3

In [17]:
# Création de la session
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name="eu-west-3"
)

# Accès au bucket existant
s3 = session.resource('s3')
bucket_name = "bucket-plan-your-trip-with-kayak"
bucket = s3.Bucket(bucket_name)

In [19]:
# Récupération des données du bucket
bucket.download_file('best_hotels.csv', 'src/best_hotels.csv')
bucket.download_file('meteo.csv', 'src/meteo.csv')

#### ETL-Transform 

In [3]:
# Création d'un dataframe avec le csv meteo
meteo = pd.read_csv('src/meteo.csv')
meteo.head()

,Id,Destination,Latitude,Longitude,Temp_J0,Rain_J0,Temp_J1,Rain_J1,Temp_J2,Rain_J2,Temp_J3,Rain_J3,Temp_J4,Rain_J4,Temp_J5,Rain_J5,Temp_J6,Rain_J6,Temp_J7,Rain_J7
0,0,Mont Saint Michel,48.635954,-1.511460,8.78,0.0,9.03,0.0,10.08,0.0,11.14,0.0,13.47,0.0,13.36,0.0,12.46,0.0,15.48,0.00
1,1,St Malo,48.649518,-2.026041,8.08,0.0,7.79,0.0,8.30,0.0,8.95,0.0,11.16,0.0,12.92,0.0,13.33,0.0,14.56,0.27
2,2,Bayeux,49.276462,-0.702474,7.45,0.0,7.29,0.0,8.50,0.0,9.28,0.0,12.76,0.0,13.99,0.0,15.07,0.0,15.74,0.00
3,3,Le Havre,49.493898,0.107973,7.94,0.0,8.31,0.0,8.76,0.0,9.56,0.0,11.17,0.0,12.29,0.0,11.57,0.0,13.08,0.00
4,4,Rouen,49.440459,1.093966,7.92,0.0,9.06,0.0,9.77,0.0,11.23,0.0,13.29,0.0,14.74,0.0,14.73,0.0,15.82,0.00


In [4]:
# Création d'un dataframe avec le csv best_hotels
hotels = pd.read_csv('src/best_hotels.csv')
hotels.head()

,destination,nom,score,latitude,longitude,url,description
0,Aigues Mortes,Boutique Hôtel des Remparts & Spa,"9,3",43.568036,4.190344,https://www.booking.com/hotel/fr/les-remparts-...,Aménagé dans une ancienne base militaire datan...
1,Aigues Mortes,La Villa Mazarin,"9,2",43.564987,4.191752,https://www.booking.com/hotel/fr/la-villa-maza...,Description de l’établissement La Villa Mazari...
2,Aigues Mortes,GOLF HOTEL,"9,1",43.568057,4.102608,https://www.booking.com/hotel/fr/golf-la-grand...,"Situé à La Grande-Motte, à 2 km de la plage de..."
3,Aigues Mortes,Hotel Les Acacias,"9,1",43.538658,4.134977,https://www.booking.com/hotel/fr/les-acacias-l...,"L'Hotel Les Acacias, de style provençal, situé..."
4,Aigues Mortes,La Maison de Lyna,"9,1",43.566018,4.192353,https://www.booking.com/hotel/fr/la-maison-de-...,L’établissement La Maison de Lyna vous accueil...


In [5]:
# Ajout d'une colonne id_destination
hotels['id_destination'] = hotels['destination'].map(meteo.set_index('Destination')['Id'])
hotels.head()

,destination,nom,score,latitude,longitude,url,description,id_destination
0,Aigues Mortes,Boutique Hôtel des Remparts & Spa,"9,3",43.568036,4.190344,https://www.booking.com/hotel/fr/les-remparts-...,Aménagé dans une ancienne base militaire datan...,25
1,Aigues Mortes,La Villa Mazarin,"9,2",43.564987,4.191752,https://www.booking.com/hotel/fr/la-villa-maza...,Description de l’établissement La Villa Mazari...,25
2,Aigues Mortes,GOLF HOTEL,"9,1",43.568057,4.102608,https://www.booking.com/hotel/fr/golf-la-grand...,"Situé à La Grande-Motte, à 2 km de la plage de...",25
3,Aigues Mortes,Hotel Les Acacias,"9,1",43.538658,4.134977,https://www.booking.com/hotel/fr/les-acacias-l...,"L'Hotel Les Acacias, de style provençal, situé...",25
4,Aigues Mortes,La Maison de Lyna,"9,1",43.566018,4.192353,https://www.booking.com/hotel/fr/la-maison-de-...,L’établissement La Maison de Lyna vous accueil...,25


#### ETL-Load

#### Connexion à la base de donnée

In [8]:
USERNAME = "postgres"  
PASSWORD = PASSWORD  
HOSTNAME = "kayak.c9au4gm0691i.eu-west-3.rds.amazonaws.com"  
DBNAME = "kayak"  
PORT = "5432"  

# Connexion à PostgreSQL avec SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DBNAME}", echo=True)

In [9]:
# Test de la connexion
with engine.connect() as connection:
    print("Connexion réussie !")

2025-03-07 18:10:06,164 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-07 18:10:06,169 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-07 18:10:06,228 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-07 18:10:06,229 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-07 18:10:06,282 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-07 18:10:06,284 INFO sqlalchemy.engine.Engine [raw sql] {}
Connexion réussie !


<img src="img/rds.png" width="900">

In [ ]:
# Envoi du DataFrame vers la table SQL meteo
meteo.to_sql(
    name="meteo", 
    con=engine,  
    index=False, 
    if_exists="replace" 
)

2025-03-07 18:10:21,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 18:10:21,430 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-03-07 18:10:21,431 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {'table_name': 'meteo', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-03-07 18:10:21,491 INFO sqlalchemy.engine.Engine 
CREATE TABLE meteo (
	"Id" BIGINT, 
	"Destination" TEXT, 
	"Latitude" FLOAT(53), 
	"Longitude" FLOAT(53), 
	"Temp_J0" FLOAT(53), 
	"Rain_J0" FLOAT(53), 
	"Temp_J1" FLOAT(53), 
	"R

35

In [11]:
# Envoi du DataFrame vers la table SQL hotels
hotels.to_sql(
    name="hotels",  
    con=engine,  
    index=False,  
    if_exists="replace"  
)

2025-03-07 18:10:47,125 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 18:10:47,128 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-03-07 18:10:47,128 INFO sqlalchemy.engine.Engine [cached since 25.7s ago] {'table_name': 'hotels', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-03-07 18:10:47,188 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotels (
	destination TEXT, 
	nom TEXT, 
	score TEXT, 
	latitude FLOAT(53), 
	longitude FLOAT(53), 
	url TEXT, 
	description TEXT, 
	id_destination BIGINT
)


2025-

700

## Top 5 des destinations

In [ ]:
# Récupération des données méteo dans la base de données
stmt = text("SELECT * "
            "FROM meteo"
            )

meteo = pd.read_sql_query(con=engine.connect(), sql=stmt)

meteo.head()

2025-03-07 18:11:23,461 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 18:11:23,463 INFO sqlalchemy.engine.Engine SELECT * FROM meteo
2025-03-07 18:11:23,463 INFO sqlalchemy.engine.Engine [generated in 0.00371s] {}


,Id,Destination,Latitude,Longitude,Temp_J0,Rain_J0,Temp_J1,Rain_J1,Temp_J2,Rain_J2,Temp_J3,Rain_J3,Temp_J4,Rain_J4,Temp_J5,Rain_J5,Temp_J6,Rain_J6,Temp_J7,Rain_J7
0,0,Mont Saint Michel,48.635954,-1.511460,8.78,0.0,9.03,0.0,10.08,0.0,11.14,0.0,13.47,0.0,13.36,0.0,12.46,0.0,15.48,0.00
1,1,St Malo,48.649518,-2.026041,8.08,0.0,7.79,0.0,8.30,0.0,8.95,0.0,11.16,0.0,12.92,0.0,13.33,0.0,14.56,0.27
2,2,Bayeux,49.276462,-0.702474,7.45,0.0,7.29,0.0,8.50,0.0,9.28,0.0,12.76,0.0,13.99,0.0,15.07,0.0,15.74,0.00
3,3,Le Havre,49.493898,0.107973,7.94,0.0,8.31,0.0,8.76,0.0,9.56,0.0,11.17,0.0,12.29,0.0,11.57,0.0,13.08,0.00
4,4,Rouen,49.440459,1.093966,7.92,0.0,9.06,0.0,9.77,0.0,11.23,0.0,13.29,0.0,14.74,0.0,14.73,0.0,15.82,0.00


In [ ]:
# Colonnes des températures
Temp_col = [col for col in meteo.columns if "Temp" in col]  
Temp_col

['Temp_J0',
 'Temp_J1',
 'Temp_J2',
 'Temp_J3',
 'Temp_J4',
 'Temp_J5',
 'Temp_J6',
 'Temp_J7']

In [16]:
# Moyenne des températures
meteo["Mean_temp"] = meteo[Temp_col].mean(axis=1).round()

In [ ]:
# Colonnes Pluies
Rain_col = [col for col in meteo.columns if "Rain" in col]  
Rain_col    

['Rain_J0',
 'Rain_J1',
 'Rain_J2',
 'Rain_J3',
 'Rain_J4',
 'Rain_J5',
 'Rain_J6',
 'Rain_J7']

In [19]:
# Nombre de jours sans pluie
meteo["Nb_day_without_rain"] = meteo[Rain_col].apply(lambda row: (row == 0).sum(), axis=1)

In [20]:
# Nombre de jours de pluie
meteo["Nb_day_with_rain"] = 8 - meteo["Nb_day_without_rain"]

In [22]:
# Total pluie
meteo['Total_rain'] = meteo[Rain_col].sum(axis=1)

In [31]:
# Moyenne de pluie quand il pleut
meteo['Mean_day_rain'] = (meteo['Total_rain'] / meteo["Nb_day_with_rain"]).round()

In [34]:
meteo['Mean_day_rain'] = meteo['Mean_day_rain'].fillna(0)

In [36]:
meteo.head()

,Id,Destination,Latitude,Longitude,Temp_J0,Rain_J0,Temp_J1,Rain_J1,Temp_J2,Rain_J2,...,Rain_J5,Temp_J6,Rain_J6,Temp_J7,Rain_J7,Mean_temp,Nb_day_without_rain,Nb_day_with_rain,Total_rain,Mean_day_rain
0,0,Mont Saint Michel,48.635954,-1.511460,8.78,0.0,9.03,0.0,10.08,0.0,...,0.0,12.46,0.0,15.48,0.00,12.0,8,0,0.00,0.0
1,1,St Malo,48.649518,-2.026041,8.08,0.0,7.79,0.0,8.30,0.0,...,0.0,13.33,0.0,14.56,0.27,11.0,7,1,0.27,0.0
2,2,Bayeux,49.276462,-0.702474,7.45,0.0,7.29,0.0,8.50,0.0,...,0.0,15.07,0.0,15.74,0.00,11.0,8,0,0.00,0.0
3,3,Le Havre,49.493898,0.107973,7.94,0.0,8.31,0.0,8.76,0.0,...,0.0,11.57,0.0,13.08,0.00,10.0,8,0,0.00,0.0
4,4,Rouen,49.440459,1.093966,7.92,0.0,9.06,0.0,9.77,0.0,...,0.0,14.73,0.0,15.82,0.00,12.0,8,0,0.00,0.0


In [83]:
# Classement
meteo_top = meteo.sort_values(by=['Nb_day_without_rain','Mean_day_rain','Mean_temp'],
               ascending=[False, True, False]).reset_index(drop=True)

In [8]:
meteo_top

,Id,Destination,Latitude,Longitude,Temp_J0,Rain_J0,Temp_J1,Rain_J1,Temp_J2,Rain_J2,...,Rain_J5,Temp_J6,Rain_J6,Temp_J7,Rain_J7,Mean_temp,Nb_day_without_rain,Nb_day_with_rain,Total_rain,Mean_day_rain
0,15,Grenoble,45.187560,5.735782,9.52,0.00,12.79,0.00,14.92,0.00,...,0.00,17.43,0.00,18.41,0.00,15.0,8,0,0.00,0.0
1,30,Toulouse,43.604462,1.444247,9.54,0.00,12.06,0.00,12.25,0.00,...,0.00,15.81,0.00,15.60,0.00,14.0,8,0,0.00,0.0
2,31,Montauban,44.017584,1.354999,9.52,0.00,11.77,0.00,13.62,0.00,...,0.00,15.87,0.00,15.52,0.00,14.0,8,0,0.00,0.0
3,0,Mont Saint Michel,48.635954,-1.511460,8.78,0.00,9.03,0.00,10.08,0.00,...,0.00,12.46,0.00,15.48,0.00,12.0,8,0,0.00,0.0
4,4,Rouen,49.440459,1.093966,7.92,0.00,9.06,0.00,9.77,0.00,...,0.00,14.73,0.00,15.82,0.00,12.0,8,0,0.00,0.0
5,5,Paris,48.853495,2.348391,8.46,0.00,9.37,0.00,9.72,0.00,...,0.00,14.73,0.00,15.27,0.00,12.0,8,0,0.00,0.0
6,6,Amiens,49.894171,2.295695,7.83,0.00,8.20,0.00,9.14,0.00,...,0.00,15.07,0.00,15.49,0.00,12.0,8,0,0.00,0.0
7,12,Besancon,47.238022,6.024362,6.56,0.00,7.85,0.00,10.33,0.00,...,0.00,15.26,0.00,16.18,0.00,12.0,8,0,0.00,0.0
8,14,Annecy,45.899235,6.128885,6.19,0.00,8.03,0.00,11.99,0.00,...,0.00,14.89,0.00,15.41,0.00,12.0,8,0,0.00,0.0
9,16,Lyon,45.757814,4.832011,4.86,0.00,7.31,0.00,10.12,0.00,...,0.00,15.66,0.00,16.19,0.00,12.0,8,0,0.00,0.0


In [41]:
# création d'une colonne pour la taille des points
meteo_top["size"] = (meteo_top["Nb_day_without_rain"] +1)**3 

In [61]:
fig = px.scatter_mapbox(meteo_top, lat="Latitude", lon="Longitude", 
						size="size", 
                        color="Mean_temp",	
                        color_continuous_scale=["blue","#947b84","red"],			
                        mapbox_style="carto-positron",
                        hover_data={"Destination": True, "Mean_temp":True, "Nb_day_with_rain":True},
                        width=800, height=700,
                        title="Meilleures Destinations en fonction de la méteo des 7 prochains jours"
                        )
fig.for_each_trace(lambda t: t.update(
    hovertemplate="<b>%{customdata[0]}</b><br>" + 
                  "T° Moyenne: %{customdata[1]}°C<br>" + 
                  "Jours de pluie: %{customdata[2]}<br>" 
    ))
fig.update_layout( mapbox_zoom=4.5)
fig.show()

In [98]:
fig = px.scatter_mapbox(meteo_top.head(), lat="Latitude", lon="Longitude", 
                        text="Destination",
						size="size", 
                        color="Mean_temp",	
                        color_continuous_scale=["blue","#947b84","red"],			
                        mapbox_style="carto-positron",
                        hover_data={"Destination": True, "Mean_temp":True, "Nb_day_with_rain":True},
                        hover_name="Destination",
                        width=800, height=700,
                        title="Top 5 des destinations en fonction de la méteo des 7 prochains jours"
                        )
fig.for_each_trace(lambda t: t.update(
    hovertemplate="<b>%{text}</b><br>" + 
                  "T° Moyenne: %{customdata[1]}°C<br>" + 
                  "Jours de pluie: %{customdata[2]}<br>" 
    ))
fig.update_layout( mapbox_zoom=5)
fig.show()

## Top 20 des hôtels de la région

In [ ]:
# Récupération des données des hôtels dans la base de données
stmt = text("SELECT * "
            "FROM hotels"
            )

hotels = pd.read_sql_query(con=engine.connect(), sql=stmt)

hotels.head()

2025-03-07 18:11:14,796 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 18:11:14,797 INFO sqlalchemy.engine.Engine SELECT * FROM hotels
2025-03-07 18:11:14,798 INFO sqlalchemy.engine.Engine [generated in 0.00226s] {}


,destination,nom,score,latitude,longitude,url,description,id_destination
0,Aigues Mortes,Boutique Hôtel des Remparts & Spa,"9,3",43.568036,4.190344,https://www.booking.com/hotel/fr/les-remparts-...,Aménagé dans une ancienne base militaire datan...,25
1,Aigues Mortes,La Villa Mazarin,"9,2",43.564987,4.191752,https://www.booking.com/hotel/fr/la-villa-maza...,Description de l’établissement La Villa Mazari...,25
2,Aigues Mortes,GOLF HOTEL,"9,1",43.568057,4.102608,https://www.booking.com/hotel/fr/golf-la-grand...,"Situé à La Grande-Motte, à 2 km de la plage de...",25
3,Aigues Mortes,Hotel Les Acacias,"9,1",43.538658,4.134977,https://www.booking.com/hotel/fr/les-acacias-l...,"L'Hotel Les Acacias, de style provençal, situé...",25
4,Aigues Mortes,La Maison de Lyna,"9,1",43.566018,4.192353,https://www.booking.com/hotel/fr/la-maison-de-...,L’établissement La Maison de Lyna vous accueil...,25


In [36]:
# score au format float
hotels['score'] = hotels['score'].apply(lambda x: float(x.replace(',','.')))

In [6]:
fig = px.scatter_mapbox(hotels, lat="latitude", lon="longitude", 
                        color="destination",			
                        mapbox_style="carto-positron",
                        hover_name="nom",
                        width=700, height=600,
                        #hover_data={"note":True},
                        title="Hôtels Recommandés"
                        )

fig.update_layout( mapbox_zoom=4.1)
fig.show()

In [60]:
fig = go.Figure()

# Meilleures destinations
fig.add_trace(
    go.Scattermapbox(
        lat=meteo_top.head()["Latitude"],
        lon=meteo_top.head()["Longitude"],
        text=meteo_top.head()["Destination"],
        mode="markers+text",
        textfont=dict(color="blue", size=12), 
        marker=go.scattermapbox.Marker(size=10, 
                                       color="blue"),
        textposition="middle right",
        hovertemplate="<b>%{text}</b> <br>Latitude: %{lat}<br>Longitude: %{lon}",
        visible=True
    )
)

visibility_list = [True]
trace_count = 1

# Meilleurs hôtels
for i in meteo_top["Id"]:
    hotel_subset = hotels[hotels["id_destination"] == i]

    fig.add_trace(
        go.Scattermapbox(
            lat=hotel_subset["latitude"],
            lon=hotel_subset["longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(size=8, 
                                           color="blue"),
            text=hotel_subset["nom"],
            textposition="top center",
            customdata=hotel_subset["score"],
            hovertemplate="<b>%{text}</b><br>Note : %{customdata}",
            visible=False
        )
    )
    visibility_list.append(False)
    trace_count += 1

# Création des boutons pour la mise à jour
buttons = [
    go.layout.updatemenu.Button(
        label="Meilleures destinations",
        method="update",
        args=[{"visible": visibility_list},
              {"mapbox.center": {"lat": meteo_top["Latitude"].mean(), "lon": meteo_top["Longitude"].mean()},
               "mapbox.zoom": 4.5}]
    )
]

for i in range(5):
    vis = [False] * trace_count
    vis[i + 1] = True
    buttons.append(
        go.layout.updatemenu.Button(
            label=f"{i+1}: " + meteo_top.loc[i, "Destination"],
            method="update",
            args=[{"visible": vis},
                  {"mapbox.center": {"lat": meteo_top.loc[i, "Latitude"], "lon": meteo_top.loc[i, "Longitude"]},
                   "mapbox.zoom": 11,
                   "title.text": f"Carte des meilleurs hôtels à {meteo_top.loc[i, 'Destination']}"},
                  ]
        )
    )

# Mise à jour de la disposition avec le centre initial
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center=dict(
            lat=meteo_top["Latitude"].mean(),
            lon=meteo_top["Longitude"].mean()
        ),
        zoom=4.5
    ),
    updatemenus=[
        go.layout.Updatemenu(
            active=0,
            buttons=buttons
        )
    ],
    title="Meilleures Destinations et Hôtels Recommandés",
    width=800,  
    height=600
)

fig.show()
